In [1]:
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
from cove import MTLSTM
from transformers import AutoTokenizer
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertModel
# Bert-large uncased (IN_CHANNELS : 300)
# hiddenlayers = 1024 (OUT_CHANNELS : 1024)

In [2]:
# Bert Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

def tokenizerfnc(str):
    return tokenizer.encode(str,max_length=512, pad_to_max_length=True)

In [3]:
TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)
TEXT_BERT = data.Field(
    use_vocab=False,
    batch_first=True,
    pad_token=tokenizer.pad_token_id,
    tokenize=tokenizerfnc
)

In [8]:
class CNN_GC(nn.Module):

    def __init__(self,out_dim=200,gc_dim=900,max_len=512, dropout=0.5):
        super(CNN_GC, self).__init__()
        
        """cove_model = CoVeEmbeddings(
            word_embeddings_dir='../model/text/stanford/glove/', 
            tokenizer=tokenizer,
            max_sequence_length=max_len, verbose=20)
        glove_model = GloVeEmbeddings()
        glove_model.load_model(dest_dir='../model/text/stanford/glove/', process=False)"""
        
        self.gc_dim = gc_dim
        self.out_dim = out_dim
        self.max_len = max_len
        self.dropout = dropout

        self.conv_3 = nn.Conv1d(gc_dim, out_dim, 3, stride=1, padding=1)
        self.conv_5 = nn.Conv1d(gc_dim, out_dim, 5, stride=1, padding=2)
        self.conv_7 = nn.Conv1d(gc_dim, out_dim, 7, stride=1, padding=3)
        self.attn = nn.Linear(3*out_dim*max_len, max_len)

    def forward(self,x):
        #cove_embed = cove_model.encode(x)
        #tokens = [sentence.split(" ") for sentence in x]
        #glove_embed = glove_model.encode(tokens)
        #x = torch.cat([cove_embed,glove_embed], 2)
        
        conv_3 = F.relu(self.conv_3(x))
        conv_5 = F.relu(self.conv_5(x))
        conv_7 = F.relu(self.conv_7(x))
        x = torch.cat([conv_3,conv_5,conv_7], 1)
        print(x.shape)
        
        non_linear_x = F.relu(x.view(-1, 600*self.max_len))
        print(non_linear_x.shape)
        attn_weights = F.softmax(self.attn(non_linear_x), dim=1)
        print(attn_weights.shape)
        #attn_applied = torch.bmm(attn_weights.unsqueeze(1), x)
        #attn_applied = attn_weights*x
        attn_applied = torch.zeros(x.shape[0], x.shape[1], x.shape[2])
        for i in range(x.shape[0]):
            attn_applied[i,:,:] = x[i,:,:]*attn_weights[i]
        print("hello")
        print(attn_applied.shape)
        
        x = attn_applied.sum(dim=2)
        return x

In [9]:
class CNN_BERT(nn.Module):

    def __init__(self, out_dim=100, embed_dim=1024, max_len=512, dropout=0.5):
        super(CNN_BERT, self).__init__()
        
        self.embed_dim = embed_dim
        self.out_dim = out_dim # not 100?
        self.max_len = max_len # ?
        self.dropout = dropout

        self.bert_layer = BertModel.from_pretrained('bert-large-uncased')
        self.conv_3 = nn.Conv1d(1, out_dim, embed_dim * 3, stride=embed_dim)
        self.conv_4 = nn.Conv1d(1, out_dim, embed_dim * 4, stride=embed_dim)
        self.conv_5 = nn.Conv1d(1, out_dim, embed_dim * 5, stride=embed_dim)

    def get_conv_out(self,conv,x, num):
        return F.max_pool1d(F.relu(conv(x)), 
                            self.max_len - num + 1).view(-1, self.out_dim)

    def forward(self,x):
        x = self.bert_layer(x)[0].view(-1, 1, self.embed_dim * self.max_len)
        conv_3 = self.get_conv_out(self.conv_3, x, 3)
        conv_4 = self.get_conv_out(self.conv_4, x, 4)
        conv_5 = self.get_conv_out(self.conv_5, x, 5)
        x = torch.cat([conv_3, conv_4, conv_5], 1)
        x = F.dropout(x, p=self.dropout)
        return x

In [10]:
# model_cnn_gc = CNN_GC()
# model_bert = CNN_BERT()

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.l0 = nn.Linear(900,300) # BERT-> CNN (300), G,C->CNN (600)
        self.l1 = nn.Linear(300,300)
        self.l3 = nn.Linear(300,1)

    def forward(self,x):
        x = torch.sigmoid(self.l0(x))
        x = torch.sigmoid(self.l1(x))
        x = torch.sigmoid(self.l3(x))
        return x


class SuggestionClassifier(nn.Module):
    def __init__(self):
        # path all the models together
        # BERT->CNN + CNN->ATT
        super(SuggestionClassifier,self).__init__()
        self.CNN_b = CNN_BERT()
        self.CNN_gc = CNN_GC()
        self.MLP = MLP()


    def forward(self,x, y):
        print("=========================")
        bert_x = self.CNN_b(x)
        print("Total Model", bert_x.shape)
        gc_x = self.CNN_gc(y)
        print("Total Model", gc_x.shape)
        out = torch.cat((bert_x,gc_x), 1) # TODO : ensure dimensions align (1x300, 1x600)
        print("Total", out.shape)
        out = self.MLP(out) # Dim : 1x900
        print("Total", out.shape)
        return out

In [12]:
model_overall = SuggestionClassifier()

In [ ]:
train_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\train_{}.csv"

for i in range(10):
    print(train_path.format(i))
    train = data.TabularDataset(
            path=train_path.format(i), format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
                    'label':('label',LABEL)
                    })

    TEXT.build_vocab(train, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
    outputs_cove_with_glove = MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

    train_iter = data.Iterator(
        (train),
        batch_size=5)
    
    for batch_idx, batch in enumerate(train_iter):
        glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
        target = torch.zeros(5, 512, 900)
        max_sentence_len_in_batch = max(batch.sentence[1].tolist())
        target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
        glove_then_last_layer_cove = target.permute(0,2,1)

        #output = model_cnn_gc(glove_then_last_layer_cove)
        # bert_output = model_bert(batch.bert_enc)
        out = model_overall(batch.bert_enc, glove_then_last_layer_cove)

        break

C:\Users\thank\Downloads\CMU MCDS\11747\project\suggestionMining\data\Subtask-A\train_0.csv
Total Model torch.Size([5, 300])
torch.Size([5, 600, 512])
torch.Size([5, 307200])
